In [8]:
def get_mov_timestamps(filename):
    ''' Get the creation and modification date-time from .mov metadata.

        Returns None if a value is not available.
    '''
    from datetime import datetime as DateTime
    import struct

    ATOM_HEADER_SIZE = 8
    # difference between Unix epoch and QuickTime epoch, in seconds
    EPOCH_ADJUSTER = 2082844800

    creation_time = modification_time = None

    # search for moov item
    with open(filename, "rb") as f:
        while True:
            atom_header = f.read(ATOM_HEADER_SIZE)
            #~ print('atom header:', atom_header)  # debug purposes
            if atom_header[4:8] == b'moov':
                break  # found
            else:
                atom_size = struct.unpack('>I', atom_header[0:4])[0]
                print(atom_size)
                f.seek(atom_size - 8, 1)

        # found 'moov', look for 'mvhd' and timestamps
        atom_header = f.read(ATOM_HEADER_SIZE)
        if atom_header[4:8] == b'cmov':
            raise RuntimeError('moov atom is compressed')
        elif atom_header[4:8] != b'mvhd':
            raise RuntimeError('expected to find "mvhd" header.')
        else:
            print(atom_header)
            f.seek(4, 1)
            creation_time = struct.unpack('>I', f.read(4))[0] - EPOCH_ADJUSTER
            creation_time = DateTime.fromtimestamp(creation_time)
            if creation_time.year < 1990:  # invalid or censored data
                creation_time = None

            modification_time = struct.unpack('>I', f.read(4))[0] - EPOCH_ADJUSTER
            modification_time = DateTime.fromtimestamp(modification_time)
            if modification_time.year < 1990:  # invalid or censored data
                modification_time = None

    return creation_time, modification_time

In [9]:
get_mov_timestamps('C:\\Users\\SHARE-LAB\\Documents\\GitHub\\GulfSouth\\samples\\IMG_E0256.MOV')

20
b'\x00\x00\x00lmvhd'


(datetime.datetime(2022, 8, 24, 18, 18, 32),
 datetime.datetime(2022, 8, 24, 18, 18, 33))

In [1]:
from io import BytesIO

from datetime import datetime as DateTime
import struct

In [52]:
def get_mov_atom(filename, atom=[b'moov',b'meta',b'keys']):
    ''' Get the creation and modification date-time from .mov metadata.

        Returns None if a value is not available.
    '''
    ATOM_HEADER_SIZE = 8
    # difference between Unix epoch and QuickTime epoch, in seconds
    EPOCH_ADJUSTER = 2082844800

    creation_time = modification_time = None
    
    cur_atom=0
    # search for moov item
    with open(filename, "rb") as f:
        while cur_atom<len(atom):
            atom_header = f.read(ATOM_HEADER_SIZE)
            atom_type = atom_header[4:8]
            atom_size = struct.unpack('>I', atom_header[0:4])[0]
            
            print('atom header:', atom_header)  # debug purposes
            if atom_type == atom[cur_atom]:
                cur_atom+=1
                # found, move to next level
            else:
                print(atom_size)
                f.seek(atom_size - ATOM_HEADER_SIZE, 1) # move current position to current offset + atom_size - ATOM_HEADER_SIZE
        
        explen=atom_size - 8
        b = f.read(explen)
        print(b)
        return read_keys(BytesIO(b), explen)

In [101]:
ATOM_HEADER_SIZE=8
EPOCH_ADJUSTER = 2082844800

def read_header(f):
    atom_header = f.read(ATOM_HEADER_SIZE)
    atom_type = atom_header[4:8]
    atom_size = struct.unpack('>I', atom_header[0:4])[0]
    return atom_type, atom_size, atom_size-ATOM_HEADER_SIZE

# def read_metas(f, hier=[b'moov',b'meta']):
#     cur_hier=0
#     num_hier=len(hier)
    
#     # search for moov item
#     with open(f, "rb") as f:
#         while cur_hier<num_hier:
#             atom_type, atom_size, body_size=read_header(f)
#             print(atom_type, atom_size)
#             if atom_type == hier[cur_hier]:
#                 # found, move to next level
#                 cur_hier+=1
#             else:
#                 # move current position to current offset + body_size
#                 f.seek(body_size, 1)
            
#         if (cur_hier==num_hier):
#             offset=0 # search for keys and values
#             exp_size=body_size
            
#             while offset<exp_size:
#                 atom_type, atom_size, body_size=read_header(f) # next atom header
#                 if atom_type == b'keys':
#                     # found, move to next level
#                     keys = read_keys(f, body_size)
#                 elif atom_type == b'ilst':
#                     items = read_values(f, body_size)
#                 else:
#                     f.seek(body_size, 1)
                
#                 offset+=atom_size
#     return {keys[i-1]:v for i, v in items}

import os
def read_metas(file, hier=[b'moov',b'meta']):
    cur_hier=0
    num_hier=len(hier)
    exp_size = os.path.getsize(file)
    
    keys=None
    items=None
    
    # search for moov item
    with open(file, "rb") as f:
        # read an atom
        offset=0
        
        while offset < exp_size-1:
            atom_type, atom_size, body_size=read_header(f)
            print(atom_size, atom_type, cur_hier, num_hier, exp_size)
            
            if cur_hier < num_hier:
                if atom_type == hier[cur_hier]:
                    # found, move to next level
                    cur_hier+=1
                    continue
                else:
                    # move current position to current offset + body_size
                    f.seek(body_size, 1)
            else:
                if atom_type == b'keys':
                    # found, move to next level
                    keys = read_keys(f, body_size)
                elif atom_type == b'ilst':
                    items = read_values(f, body_size)
                else:
                    # move current position to current offset + body_size
                    f.seek(body_size, 1)
                if (keys!=None and items != None):
                    break
                    
            offset+=atom_size;
            
    return {keys[i-1]:v for i, v in items}

def read_keys(f, exp_size, atom=b'mdta'):
    '''
    f: file object
    exp_size: expected bytes to read (as a validation process)
    '''
    keys=[]
    i_key = 0
    
    # version (1 byte) + flag (3 byte) + size (4 bytes)
    # see "Metadata Item Keys Atom"
    # in https://developer.apple.com/library/archive/documentation/QuickTime/QTFF/Metadata/Metadata.html
    
    offset = 8
    num_keys = struct.unpack('>I', f.read(offset)[4:8])[0] 

    while i_key < num_keys:
        atom_type, atom_size, body_size=read_header(f)
        atom_data = f.read(body_size).decode()
        keys.append(atom_data)

        i_key+=1
        offset += atom_size

    assert offset==exp_size
    
    return keys

def read_values(f, exp_size, atom=b'mdta'):
    '''
    f: file object
    exp_size: expected bytes to read (as a validation process)
    '''
    offset = 0
    values=[]
    while offset<exp_size:
        dontknow = struct.unpack('>I', f.read(4))[0]
        index = struct.unpack('>I', f.read(4))[0] 
        atom_type, atom_size, body_size=read_header(f)
        
        dtype=struct.unpack('>I', f.read(4))[0]
        loc=struct.unpack('>I', f.read(4))[0]
        data=f.read(body_size-8).decode()
        
        offset+=(atom_size+8)
        
        values.append([index, data])
        print(dontknow, index, dtype, loc)

    return values

In [102]:
read_metas('C:\\Users\\SHARE-LAB\\Documents\\GitHub\\GulfSouth\\samples\\IMG_5233.MOV')

20 b'ftyp' 0 2 235033859
8 b'wide' 0 2 235033859
234963574 b'mdat' 0 2 235033859
70257 b'moov' 0 2 235033859
108 b'mvhd' 1 2 235033859
24221 b'trak' 1 2 235033859
25078 b'trak' 1 2 235033859
629 b'trak' 1 2 235033859
2278 b'trak' 1 2 235033859
17402 b'trak' 1 2 235033859
533 b'meta' 1 2 235033859
34 b'hdlr' 2 2 235033859
257 b'keys' 2 2 235033859
234 b'ilst' 2 2 235033859
32 1 1 0
50 2 1 0
29 3 1 0
37 4 1 0
30 5 1 0
48 6 1 0


{'com.apple.quicktime.location.accuracy.horizontal': '4.746088',
 'com.apple.quicktime.location.ISO6709': '+29.6522-082.3407+049.240/',
 'com.apple.quicktime.make': 'Apple',
 'com.apple.quicktime.model': 'iPhone 12 Pro',
 'com.apple.quicktime.software': '15.6.1',
 'com.apple.quicktime.creationdate': '2022-08-29T17:33:18-0400'}